# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.3.2 with hadoop 3.3, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
    !tar xf spark-3.3.2-bin-hadoop3.tgz
    !mv spark-3.3.2-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

In [4]:
# I run it on local
import os
os.environ["SPARK_HOME"] = "/opt/homebrew/Cellar/apache-spark/3.5.1/libexec"

# Start a Local Cluster

In [5]:
import findspark
findspark.init()

In [6]:
spark_url = 'local'

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark Assignment')\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

24/04/09 12:47:43 WARN Utils: Your hostname, Titors-MacBook-Pro-4TB-32GB.local resolves to a loopback address: 127.0.0.1; using 10.203.163.61 instead (on interface en0)
24/04/09 12:47:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/09 12:47:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/04/09 12:48:00 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


# Spark Assignment

Based on the movie review dataset in 'netflix-rotten-tomatoes-metacritic-imdb.csv', answer the below questions.

**Note:** do not clean or remove missing data

In [8]:
sc = spark.sparkContext

In [9]:
spark

In [10]:
sc

<SparkContext master=local appName=Spark Assignment>

In [13]:
data_path = './netflix-rotten-tomatoes-metacritic-imdb.csv'

In [14]:
df = spark.read.csv(data_path, header=True, inferSchema=True)

## What is the maximum and average of the overall hidden gem score?

In [17]:
from pyspark.sql.functions import max, avg

hidden_gem_stats = df.agg(
    max("Hidden Gem Score").alias("Maximum Hidden Gem Score"),
    avg("Hidden Gem Score").alias("Average Hidden Gem Score")
)

hidden_gem_stats.show()

+------------------------+------------------------+
|Maximum Hidden Gem Score|Average Hidden Gem Score|
+------------------------+------------------------+
|                     9.8|       5.937551386501226|
+------------------------+------------------------+



## How many movies that are available in Korea?

In [24]:
from pyspark.sql.functions import col

korea_movie_count = df.filter(col("Languages").contains("Korea")).count()
print(f"Number of movies available in Korea: {korea_movie_count}")

Number of movies available in Korea: 735


## Which director has the highest average hidden gem score?

In [20]:
from pyspark.sql.functions import avg, col

director_avg_scores = df.groupBy("Director") \
                         .agg(avg("Hidden Gem Score").alias("Average Hidden Gem Score")) \
                         .orderBy(col("Average Hidden Gem Score").desc())

top_director = director_avg_scores.first()

print(f"Director with the highest average hidden gem score: {top_director['Director']} (Score: {top_director['Average Hidden Gem Score']})")

Director with the highest average hidden gem score: Dorin Marcu (Score: 9.8)


## How many genres are there in the dataset?

In [21]:
from pyspark.sql.functions import explode, split, col

genre_df = df.withColumn("Genre", explode(split(col("Genre"), ",\s*")))
genre_count = genre_df.select("Genre").distinct().count()

print(f"Total number of unique genres: {genre_count}")

Total number of unique genres: 28
